# The Two-Rigid-Rod-Problem

The problem is taken from a series of exercises in a numerical analysis course by Prof. John H. Maddocks.
In particular here: https://lcvmwww.epfl.ch/~lcvm/dna_teaching_05_06/exercises/sol5.pdf

The system consists of two rigid rods connected to the floor and each other by spring-loaded joints.
Its configuration is described by two solution variables $\theta$ and $\phi$ giving the angle between each rod and the vertical. 

The bottom spring has the neutral position straight up ($\theta=0$) and the spring between the two rods has the neutral 
position when the rods for a straight line ($\phi = \theta$). At the end of the second rod a load $\lambda\geq 0$ is added, pointing straight down.

The interactive graphic below illustrates the configuration. It is completely unconstraied, i.e. the configurations shown are not necessarily solutions to the equilibrum problem.

In [1]:
from ipywidgets import FloatSlider, VBox, HBox, HTML
from math import pi
from tworod import draw_solution_svg

solution = HTML()
theta = FloatSlider(value=0.49082376, min=-pi, max=pi, step=1e-4, description='theta')
phi = FloatSlider(value=0.78532203, min=-pi, max=pi, step=1e-4, description='phi')
lam = FloatSlider(min=0.41651516, max=5, step=1e-4, description='lambda')

def update_solution(*_unused):
    svg = draw_solution_svg((theta.value, phi.value), lam.value)
    solution.value = (f'<div style="max-width: 500px">{svg}</div>')
update_solution()

theta.observe(update_solution, 'value')
phi.observe(update_solution, 'value')
lam.observe(update_solution, 'value')

VBox([solution, theta, phi, lam])

## Finding Equilibrium Solutions

We are interested in finding equilibrium solutions for this system, for various loads $\lambda$. 
Without any load the solution $\theta = \phi = 0$ is an equilibrium solution. In fact the springs in the joints make it a stable equilibrium. This is the solution from which we will use parameter continuation to find other solutions.

The continuation itself happens in the `branch.extend` calls below.

In [2]:
from continuation import BifurcationAnalysis, BifWidget
import tworod

bif = BifurcationAnalysis(tworod.F, tworod.J, tworod.F_lam, (0,0), 0)
bif.branches[0].extend(6)

The following code creates an interactive diagram. On the left you can see the bifurcation diagram. By clicking a point on the bifurcation diagram the corresponding solution is displayed below. On the right you can see a heatmap of the potential for the given $\lambda$. We will discuss the various branches below. The axes on the bifurcation diagram are the load $\lambda$ and the first angle $\theta$. The solution space is not planar, so we have to make a choice of the variables shown. The branches go in fact in different directions in the $(\theta, \phi)$ space.

In [3]:
%%capture --no-stdout --no-display
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
fig, [ax, ax_field] = plt.subplots(nrows=1, ncols=2, figsize=(8, 4))
fig.canvas.header_visible = False
ax.set_xlabel('λ'); ax.set_ylabel('θ'); ax.set_title('Bifurcation Diagram')
ax_field.set_xlabel('θ'); ax_field.set_ylabel('φ'); ax_field.set_title('Potential')
cursor, = ax.plot([0],[0],'o')
for i, b in enumerate(bif.branches):
    line, = ax.plot(b.param, b.soln[:,0], picker=5);
    line.branch_index = i
    line.branch = b
out = HTML('out')

def on_pick(event):
    pick_event = event
    ind = event.ind[len(event.ind)//2]
    soln = event.artist.branch.soln[ind]
    param = event.artist.branch.param[ind]
    out.value = f'{ind}, {event.artist.branch_index}: {soln}'
    update_graphics(soln, param)

def update_graphics(soln, param):
    update_svg(soln, param)
    update_vectorfield(soln, param)
    
def update_svg(soln, param):
    svg = draw_solution_svg(soln, param)
    J = bif.J(soln, param)
    [r1, r2], [v1, v2] = np.linalg.eig(J)
    out.value = (f'<p>J = {J}</p>'
                 f'<p>Eigenvalue {r1:0.3f}: {v1[0]:0.3f}, {v1[1]:0.3f}</p>'
                 f'<p>Eigenvalue {r2:0.3f}: {v2[0]:0.3f}, {v2[1]:0.3f}</p>'
                 f'<div style="max-width: 600px">{svg}</div>')

def update_vectorfield(soln, param):
    ax_field.clear()
    ax_field.set_xlabel('θ'); ax_field.set_ylabel('φ'); ax_field.set_title('Potential')
    HALF_BOX = 3.2
    g = np.meshgrid(np.linspace(-HALF_BOX,HALF_BOX,50),np.linspace(-HALF_BOX,HALF_BOX,50))
    x,y = g
    z = tworod.E(g, param)
    #z = np.log(z + 1 - np.min(z))
    # x and y are bounds, so z should be the value *inside* those bounds.
    # Therefore, remove the last value from the z array.
    z = z[:-1, :-1]
    z_min, z_max = z.min(), z.max()
    ax_field.pcolormesh(x, y, z, cmap='coolwarm', vmin=z_min, vmax=z_max)
    g = np.meshgrid(np.linspace(-HALF_BOX,HALF_BOX,11),np.linspace(-HALF_BOX,HALF_BOX,11))
    w = -bif.F(g, param)
    w /= np.linalg.norm(w, axis=0)
    u,v = w
    x,y = g
    ax_field.plot(soln[0],soln[1],'o', color='lightgreen', mec='black')
    ax_field.quiver(x,y,u,v)
    
    cursor.set_xdata([param])
    cursor.set_ydata([soln[0]])
    
update_graphics((0,0), 0)
cid = fig.canvas.mpl_connect('pick_event', on_pick)
out

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

HTML(value='<p>J = [[ 2. -1.]\n [-1.  1.]]</p><p>Eigenvalue 2.618: 0.851, 0.526</p><p>Eigenvalue 0.382: -0.526…

The bifurcation diagram consists of 5 branches. 
* The horizontal axis corresponds to the straight up solutions ($\theta = \phi = 0$). For small loads this is stable, for higher loads it is an unstable equilibrium.
  * Until the first bifurcation point this is a stable equilibrium, the springs are strong enough to hold the load.
  * After the first bifurcation point it becomes an unstable solution, first a saddle with one negative eigenvalue.
  * After the second bifurcation it is a local maximum in the potential with two negative eigenvalues.

* The two branches starting at the first bifurcation point correspond to solution where the rods lean to one side, 
  i.e. $\theta$ and $\phi$ have the same sign. They are stable equilibria for higher loads.
  
* The two remaining branches starting at the second bifurcation point correspond to the rod folding in on itself.
  They are unstable (saddle) equilibria. The signs of $\theta$ and $\phi$ are opposite.